# Replication of Main Figures / Tables

**Note**: See `README.md` in the main folder if you have not already, for instructions on how to generate the experiment results that this notebook uses.

This notebook will replicate selected figures / tables in the main paper.

In [ ]:
# This flag can be set to use the same hyperparameters and thresholds 
# as in our published work.  Due to differences in features (described in README.md), there are still
# some minor differences, but this will more closely replicate our published results
USE_REP_HP = False

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('white')
sns.set_palette('deep')

plt.rcParams.update({'font.size': 20})
plt.rcParams['ps.useafm'] = True
plt.rcParams['pdf.use14corefonts'] = True
plt.rcParams['text.usetex'] = False

import os

REPO_PATH = os.environ['REPO_PATH']
EXP_PATH = os.environ['EXP_RESULT_PATH']
DIRECT_EXP_PATH = f"{EXP_PATH}/direct/direct_learning_test"

if USE_REP_HP:
    ER_EXP_PATH = f"{EXP_PATH}/expected_reward/expected_reward_eval_test_rep/results"
    THRESH_EXP_PATH = f"{EXP_PATH}/thresholding/thresholding_eval_test_rep/results"
else:
    ER_EXP_PATH = f"{EXP_PATH}/expected_reward/expected_reward_eval_test/results"
    THRESH_EXP_PATH = f"{EXP_PATH}/thresholding/thresholding_eval_test/results"

In [ ]:
# Modify file paths here as needed
direct_frontier = pd.read_csv(f"{DIRECT_EXP_PATH}/frontier_test.csv")
exp_reward_frontier = pd.read_csv(f"{ER_EXP_PATH}/frontier_test.csv")
thresholding_frontier = pd.read_csv(f"{THRESH_EXP_PATH}/best_test_outcomes_by_max_broad.csv")

In [ ]:
exp_reward_frontier = exp_reward_frontier.query("omega >= 0.85")
direct_frontier = direct_frontier.query("param >= 0.85")

### Figure 1

To get the values of the Yelin baseline, run the following

```bash
cd ${REPO_PATH}/experiments
bash scripts/eval_test_scripts/test_yelin_baseline.sh
```

Or if you are running with replication hyperparameters, use the following

```bash
cd ${REPO_PATH}/experiments
bash scripts/eval_test_scripts/test_yelin_baseline_rep.sh
```

In [ ]:
# Yellin Baseline
yellin = {
    'constrained': [0.10910936310581071,0.33570159857904086],
    'unconstrained': [0.054554681552905354,0.965237249429079]
}

In [ ]:
plt.figure(figsize=(10,8))

# Our methods
plt.scatter(direct_frontier['iat'],
           direct_frontier['broad'],
           label='Direct', 
           s=60)

plt.scatter(exp_reward_frontier['iat'],
           exp_reward_frontier['broad'],
           label='Expected reward', marker="D", s=60)

plt.scatter(thresholding_frontier['iat_prop'],
           thresholding_frontier['broad_prop'],
           label='Thresholding', s=60, marker="^")


# Clinicians
plt.scatter([0.119], [0.336],
          label='Doctor', s=180, marker="P")

# Constraint Yelin result
plt.scatter(yellin['constrained'][0], yellin['constrained'][1],
            label='Constrained [31]', s=150, marker="s",
           color='c')

# Unconstrained Yelin result
plt.scatter(yellin['unconstrained'][0], yellin['unconstrained'][1],
            label='Unconstrained [31]',
            s=250, marker="X", color='m')

plt.xlim(0.05, 0.122)

plt.xlabel("IAT rate")
plt.ylabel("2nd-line usage")

plt.legend(fontsize=18);

## Tables 2, 3

In [ ]:
def get_values_const_2line(frontier, threshold):
    iat = frontier.query('broad < @threshold').sort_values(by='broad', ascending=False).iloc[0]['iat']
    broad = frontier.query('broad < @threshold').sort_values(by='broad', ascending=False).iloc[0]['broad']  
    return {'IAT': iat, '2nd-line Usage': broad}

table_2_dict = {
    'Doctor': {'IAT': 0.119, '2nd-line Usage': 0.336},
    'Thresholding': get_values_const_2line(thresholding_frontier.rename(
        columns={'broad_prop': 'broad', 'iat_prop': 'iat'}), 0.336),
    'Expected Reward': get_values_const_2line(exp_reward_frontier, 0.336),
    'Direct Learning': get_values_const_2line(direct_frontier, 0.336),
}

print(pd.DataFrame.from_dict(table_2_dict).T)

In [ ]:
def get_values_improve(frontier, threshold):
    iat = frontier.query('iat < @threshold').sort_values(by='iat', ascending=False).iloc[0]['iat']
    broad = frontier.query('iat < @threshold').sort_values(by='iat', ascending=False).iloc[0]['broad']  
    return {'IAT': iat, '2nd-line Usage': broad}

table_3_dict = {
    'Doctor': {'IAT': 0.119, '2nd-line Usage': 0.336},
    'Thresholding': get_values_improve(thresholding_frontier.rename(
        columns={'broad_prop': 'broad', 'iat_prop': 'iat'}), 0.119),
    'Expected Reward': get_values_improve(exp_reward_frontier, 0.119),
    'Direct Learning': get_values_improve(direct_frontier, 0.119),
}

print(pd.DataFrame.from_dict(table_3_dict).T)

### Figure 2

In [ ]:
# Modify file paths here as needed
test_results_deferral = pd.read_csv(f"{EXP_PATH}/direct/direct_learning_defer_interventions_test/frontier_test.csv")

In [ ]:
plt.figure(figsize=(16,4))

plt.subplot(121)

plt.scatter(test_results_deferral['defer_rate'],
           test_results_deferral['iat_decision'],
           label='Algorithm', s=100)

plt.scatter(test_results_deferral['defer_rate'],
           test_results_deferral['iat_doc'],
           label='Doctor', s=150, marker="P")
plt.xlabel("Deferral rate")
plt.ylabel("IAT rate")
plt.title("IAT rate comparison: Decision cohort",
         fontsize=22)
plt.ylim(0.08, 0.14)

plt.subplot(122)

plt.scatter(test_results_deferral['defer_rate'],
           test_results_deferral['broad_decision'],
           label='Algorithm', s=100)

plt.scatter(test_results_deferral['defer_rate'],
           test_results_deferral['broad_doc'],
           label='Doctor', s=150, marker="P")

plt.xlabel("Deferral rate")
plt.ylabel("2nd-line usage rate")
plt.title("2nd-line usage comparison: Decision cohort",
         fontsize=22)

plt.legend(fontsize=18, bbox_to_anchor=(1.45, .7))
plt.ylim(.0, 0.5)
plt.show()